# Capstone project notebook
*This* capstone project notebook is the notebook containing the code and analysis for the final **capstone project**: 
> ## Battle of the neighbourhoods

In [1]:
# import required libraries
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


---
## Assignment week 3
> ### Segmenting and Clustering Neighborhoods in Toronto 

### *Task 1*: Building the dataframe

Importing the required libraries

In [112]:
# import required libraries
import pandas as pd
import numpy as np
#!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

Reading in the borough vs. neighboorhood data from wikipedia, and restructuring the data frame as per the instructions.

In [113]:
# read table from wikipedia
df_toro=pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641')[0]
df_toro.head(10)
# skip entries with non-assigned buroughs
df_toro.drop(df_toro[df_toro['Borough']=='Not assigned'].index, inplace=True)
# make non-assigned neighbourhood names the same
df_toro.loc[df_toro[df_toro['Neighbourhood']=='Not assigned'].index, 'Neighbourhood']=df_toro[df_toro['Neighbourhood']=='Not assigned']['Borough']
# merge neighbourhoods with same postal code
neigh_acc=df_toro.groupby('Postcode')['Neighbourhood'].apply(', '.join)
df_toro_filt=pd.merge(df_toro.drop(columns='Neighbourhood'), neigh_acc, left_on='Postcode', right_on='Postcode', how='inner')
df_toro_filt.drop_duplicates(inplace=True)
df_toro_filt.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge, Malvern"
10,M3B,North York,Don Mills North
11,M4B,East York,"Woodbine Gardens, Parkview Hill"
13,M5B,Downtown Toronto,"Ryerson, Garden District"


In [114]:
# size of the dataframe
df_toro_filt.shape

(103, 3)

Add coordinates to dataframe

In [115]:
# initialize 
latitude=[]
longitude=[]
# loop through postal codes in dataframe
for postal_code in df_toro_filt['Postcode']:
    print(postal_code)
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        print(lat_lng_coords)
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])

M3A
[43.75245000000007, -79.32990999999998]
M4A
[43.73057000000006, -79.31305999999995]
M5A
[43.65512000000007, -79.36263999999994]
M6A
[43.72327000000007, -79.45041999999995]
M7A
[43.66253000000006, -79.39187999999996]
M9A
[43.662630000000036, -79.52830999999998]
M1B
[43.811390000000074, -79.19661999999994]
M3B
[43.74923000000007, -79.36185999999998]
M4B
[43.70718000000005, -79.31191999999999]
M5B
[43.65739000000008, -79.37803999999994]
M6B
[43.70687000000004, -79.44811999999996]
M9B
[43.65034000000003, -79.55361999999997]
M1C
[43.78574000000003, -79.15874999999994]
M3C
[43.72168000000005, -79.34351999999996]
M4C
[43.68970000000007, -79.30681999999996]
M5C
[43.65215000000006, -79.37586999999996]
M6C
[43.69211000000007, -79.43035999999995]
M9C
[43.64857000000006, -79.57824999999997]
M1E
[43.765750000000025, -79.17469999999997]
M4E
[43.67709000000008, -79.29546999999997]
M5E
[43.64536000000004, -79.37305999999995]
M6E
[43.68784000000005, -79.45045999999996]
M1G
[43.76812000000007, -79.2

In [116]:
# add coordinates to dataframe
df_toro_all=df_toro_filt
df_toro_all['Latitude']=latitude
df_toro_all['Longitude']=longitude
df_toro_all.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.65512,-79.36264
4,M6A,North York,"Lawrence Heights, Lawrence Manor",43.72327,-79.45042
6,M7A,Queen's Park,Queen's Park,43.66253,-79.39188
7,M9A,Etobicoke,Islington Avenue,43.66263,-79.52831
8,M1B,Scarborough,"Rouge, Malvern",43.81139,-79.19662
10,M3B,North York,Don Mills North,43.74923,-79.36186
11,M4B,East York,"Woodbine Gardens, Parkview Hill",43.70718,-79.31192
13,M5B,Downtown Toronto,"Ryerson, Garden District",43.65739,-79.37804


#### Summary of the dataframe

In [117]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toro_all['Borough'].unique()),
        df_toro_all.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


### *Task 2:* Explore the neighbourhoods

importing additional libraries

In [107]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Marc\anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0




geopy-2.0.0          | 63 KB     |            |   0% 
geopy-2.0.0          | 63 KB     | ##5        |  25% 
geopy-2.0.0          | 63 KB     | ########## | 100% 
geopy-2.0.0          | 63 KB     | ########## | 100% 

geographiclib-1.50   

Get the coordinates of Toronto for visualization in Folium.

In [125]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ToCa_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create a map of Toronto with markers for Neighbourhoods

In [131]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toro_all['Latitude'], df_toro_all['Longitude'], df_toro_all['Borough'], df_toro_all['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
display(map_toronto)

Define Foursquare credentials for later use to explore the neighbourhoods

In [129]:
CLIENT_ID = 'FSIO1MYBM1ZNXJOTFLFTZPKKGB2120LOAZ0GLCQACIU5YLSK' # your Foursquare ID
CLIENT_SECRET = 'RGVX3WRVVVNAC3PMISJTW2CAPTZE0MWSRWNAJIRCJANFCUPJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FSIO1MYBM1ZNXJOTFLFTZPKKGB2120LOAZ0GLCQACIU5YLSK
CLIENT_SECRET:RGVX3WRVVVNAC3PMISJTW2CAPTZE0MWSRWNAJIRCJANFCUPJ


Get nearby venues for each neighbourhood

In [130]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get top 100 venues for each neighbourhood

In [132]:
# type your answer here
toronto_venues=getNearbyVenues(df_toro_all['Neighbourhood'], df_toro_all['Latitude'], df_toro_all['Longitude'], radius=500)

Parkwoods
Victoria Village
Harbourfront, Regent Park
Lawrence Heights, Lawrence Manor
Queen's Park
Islington Avenue
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The D

In [137]:
toronto_venues.head(30)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75245,-79.32991,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75245,-79.32991,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.73057,-79.31306,Wigmore Park,43.731023,-79.310771,Park
3,Victoria Village,43.73057,-79.31306,Memories of Africa,43.726602,-79.312427,Grocery Store
4,Victoria Village,43.73057,-79.31306,Guardian Drug,43.730584,-79.307432,Pharmacy
5,"Harbourfront, Regent Park",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
6,"Harbourfront, Regent Park",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
7,"Harbourfront, Regent Park",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
8,"Harbourfront, Regent Park",43.65512,-79.36264,Berkeley Church,43.655123,-79.365873,Event Space
9,"Harbourfront, Regent Park",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio


How many venues per neighborhood were returned

In [140]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,16,16,16,16,16,16
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",16,16,16,16,16,16
"Alderwood, Long Branch",4,4,4,4,4,4
...,...,...,...,...,...,...
Willowdale West,5,5,5,5,5,5
Woburn,4,4,4,4,4,4
"Woodbine Gardens, Parkview Hill",15,15,15,15,15,15


One hot encoding to convert venue type into vector

In [145]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(20)

,Zoo Exhibit,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Group rows by neighbourhood and convert to frequency of occurence per category of venue and neighbourhood

In [146]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo Exhibit,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.03,0.0,0.0,0.01,0.0,0.000000,...,0.01,0.0,0.0,0.0000,0.0000,0.01,0.0,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,...,0.00,0.0,0.0,0.0000,0.0625,0.00,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,...,0.00,0.0,0.0,0.0000,0.0000,0.00,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,...,0.00,0.0,0.0,0.0625,0.0000,0.00,0.0,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,...,0.00,0.0,0.0,0.0000,0.0000,0.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Willowdale West,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,...,0.00,0.0,0.0,0.0000,0.0000,0.00,0.0,0.0,0.0,0.0
96,Woburn,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,...,0.00,0.0,0.0,0.0000,0.0000,0.00,0.0,0.0,0.0,0.0
97,"Woodbine Gardens, Parkview Hill",0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,...,0.00,0.0,0.0,0.0000,0.0000,0.00,0.0,0.0,0.0,0.0
98,Woodbine Heights,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.055556,...,0.00,0.0,0.0,0.0000,0.0000,0.00,0.0,0.0,0.0,0.0


Display top 5 venue per neighbourhood

In [147]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.07
1        Hotel  0.06
2         Café  0.06
3   Restaurant  0.04
4          Gym  0.04


----Agincourt----
                 venue  freq
0   Chinese Restaurant  0.12
1      Bubble Tea Shop  0.06
2        Shopping Mall  0.06
3  Shanghai Restaurant  0.06
4               Bakery  0.06


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
              venue  freq
0  Sushi Restaurant  0.33
1          Pharmacy  0.33
2      Intersection  0.33
3       Zoo Exhibit  0.00
4         Nightclub  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.12
1        Hardware Store  0.06
2   Japanese Restaurant  0.06
3  Fast Food Restaurant  0.06
4                  Park  0.06


----Alderwood, Long Branch----
                   venue  freq
0      Convenience Store  0.25
1                    Pub  0.2

4            Museum  0.00


----East Toronto----
               venue  freq
0  Convenience Store  0.33
1               Park  0.33
2       Intersection  0.33
3             Museum  0.00
4       Noodle House  0.00


----Emery, Humberlea----
         venue  freq
0  Coffee Shop  0.50
1    Nightclub  0.25
2         Park  0.25
3  Zoo Exhibit  0.00
4  Opera House  0.00


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.11
1         Women's Store  0.05
2           Coffee Shop  0.05
3  Fast Food Restaurant  0.05
4            Restaurant  0.04


----First Canadian Place, Underground city----
         venue  freq
0  Coffee Shop  0.12
1        Hotel  0.08
2         Café  0.06
3   Restaurant  0.04
4          Gym  0.04


----Flemingdon Park, Don Mills South----
             venue  freq
0  Bubble Tea Shop  0.12
1       Smoke Shop  0.12
2       Beer Store  0.12
3      Coffee Shop  0.12
4    Grocery Store  0.12


----Forest Hill North, Forest Hill West----
   

                       venue  freq
0             Sandwich Place  0.14
1                       Café  0.10
2                Pizza Place  0.05
3         Mexican Restaurant  0.05
4  Middle Eastern Restaurant  0.05


----The Beaches----
               venue  freq
0                Pub  0.25
1  Health Food Store  0.25
2              Trail  0.25
3        Zoo Exhibit  0.00
4      Moving Target  0.00


----The Beaches West, India Bazaar----
                  venue  freq
0                  Park  0.11
1     Fish & Chips Shop  0.06
2           Coffee Shop  0.06
3            Steakhouse  0.06
4  Fast Food Restaurant  0.06


----The Danforth West, Riverdale----
              venue  freq
0    Ice Cream Shop  0.12
1          Bus Line  0.12
2      Intersection  0.12
3              Park  0.12
4  Business Service  0.12


----The Junction North, Runnymede----
                        venue  freq
0      Furniture / Home Store   0.4
1          Seafood Restaurant   0.2
2            Business Service   0.2
3     

Make a dataframe with top 10 venues per neighbourhood

In [150]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [151]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Hotel,Café,Gym,Restaurant,Japanese Restaurant,Salad Place,American Restaurant,Steakhouse,Asian Restaurant
1,Agincourt,Chinese Restaurant,Shanghai Restaurant,Supermarket,Bakery,Bubble Tea Shop,Badminton Court,Discount Store,Newsagent,Sushi Restaurant,Department Store
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Sushi Restaurant,Intersection,Yoga Studio,Event Space,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Falafel Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Discount Store,Sandwich Place,Fried Chicken Joint,Beer Store,Fast Food Restaurant,Japanese Restaurant,Liquor Store,Coffee Shop,Caribbean Restaurant
4,"Alderwood, Long Branch",Performing Arts Venue,Convenience Store,Pub,Gym,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room


key means clustering of neighbourhoods by top 10 venues and visualization of the clusters in folium

In [174]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_
toronto_grouped.shape

(100, 270)

In [177]:
# add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toro_all

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighborhood'), on='Neighbourhood', how='right')
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int) 
toronto_merged.head() # check the last columns!
 

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.75245,-79.32991,2,Food & Drink Shop,Park,Yoga Studio,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Farm
1,M4A,North York,Victoria Village,43.73057,-79.31306,0,Grocery Store,Pharmacy,Park,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Yoga Studio
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.65512,-79.36264,0,Coffee Shop,Breakfast Spot,Yoga Studio,Italian Restaurant,Pub,Restaurant,Distribution Center,Electronics Store,Event Space,Food Truck
4,M6A,North York,"Lawrence Heights, Lawrence Manor",43.72327,-79.45042,0,Clothing Store,Furniture / Home Store,Cosmetics Shop,Bookstore,Men's Store,Toy / Game Store,Women's Store,Food Court,American Restaurant,Coffee Shop
6,M7A,Queen's Park,Queen's Park,43.66253,-79.39188,0,Coffee Shop,Sandwich Place,Café,Gastropub,Theater,Fried Chicken Joint,Italian Restaurant,Bank,Mediterranean Restaurant,Park


#### Clustering results - visualization & observations

In [194]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
display(map_clusters)

Cluster 1 - coffeeshops, groceries, restaurants and hotels - touristy / going-out neighbourhoods

In [184]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Grocery Store,Pharmacy,Park,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Yoga Studio
2,Downtown Toronto,0,Coffee Shop,Breakfast Spot,Yoga Studio,Italian Restaurant,Pub,Restaurant,Distribution Center,Electronics Store,Event Space,Food Truck
4,North York,0,Clothing Store,Furniture / Home Store,Cosmetics Shop,Bookstore,Men's Store,Toy / Game Store,Women's Store,Food Court,American Restaurant,Coffee Shop
6,Queen's Park,0,Coffee Shop,Sandwich Place,Café,Gastropub,Theater,Fried Chicken Joint,Italian Restaurant,Bank,Mediterranean Restaurant,Park
7,Etobicoke,0,Pharmacy,Park,Grocery Store,Café,Shopping Mall,Skating Rink,Bank,Event Space,Falafel Restaurant,Dumpling Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
191,Downtown Toronto,0,Coffee Shop,Hotel,Café,Gym,Restaurant,American Restaurant,Asian Restaurant,Japanese Restaurant,Deli / Bodega,Seafood Restaurant
196,Downtown Toronto,0,Coffee Shop,Restaurant,Japanese Restaurant,Gay Bar,Sushi Restaurant,Café,Men's Store,Dance Studio,Hotel,Bubble Tea Shop
197,East Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Italian Restaurant,Bar,Asian Restaurant,Theater,Sandwich Place,Salon / Barbershop
198,Etobicoke,0,Chinese Restaurant,Park,Fast Food Restaurant,Sushi Restaurant,Bank,Italian Restaurant,Coffee Shop,Flower Shop,Deli / Bodega,Electronics Store


Cluster 2 - contains auto garage

In [187]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
127,Scarborough,1,Auto Garage,Yoga Studio,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Farm,Dumpling Restaurant


Cluster 3 - Parks, Stores, Schools, Electronics Store - Residential areas

In [192]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Food & Drink Shop,Park,Yoga Studio,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Farm
44,North York,2,Residential Building (Apartment / Condo),Park,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Yoga Studio
82,North York,2,Park,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Donut Shop
89,North York,2,Bakery,Park,Yoga Studio,Event Space,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Dumpling Restaurant
122,Central Toronto,2,French Restaurant,Park,Yoga Studio,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Farm


Cluster 4 - Neighbourhoods with trails for Hiking

In [190]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,East Toronto,3,Health Food Store,Pub,Trail,Yoga Studio,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room
43,Scarborough,3,Trail,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Donut Shop
150,Central Toronto,3,Playground,Gym,Trail,Yoga Studio,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room


Cluster 5 - has a swimming pool (and a fish market)

In [193]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
193,Etobicoke,4,Pool,Donut Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
